In [1]:
# Installing required libraries
!pip install numpy pandas faiss-gpu torch transformers sentence_transformers rouge --quiet
!pip install rouge-score
!pip install simplet5
!pip install wikipedia

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 276.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=36ad3bffd00511f510b2a4eb84804846a7f3a322e82888ee213aa957114f662d
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 4.6 MB/s eta 0:00:00
Requested pytorch-lightning==1.5.10 from https://files.pythonhosted.org/packages/18/f1/f59b307f75db1886c96e396eec878501510677394868680b8d2b8b58c47c/pytorch_lightning-1.5.10-py3-none-any.whl (from simplet5) has invalid metadata: .* suffix can only be used with `==` or `!=` operators
    torch (>=1.7.*)
           ~~~~~~^
Please use pip<24.1 if you need to use this version.
INFO:

In [5]:
#Importing all the libraries required
import re
import numpy as np
import pandas as pd
import faiss
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from transformers import BartForConditionalGeneration, BartTokenizer, T5Tokenizer, T5ForConditionalGeneration, PegasusTokenizer, PegasusForConditionalGeneration
from rouge import Rouge
from rouge_score import rouge_scorer
import wikipedia
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import sent_tokenize
import textwrap

In [6]:
#Reading the dataset(Initial-MovieDataset1)
movies = pd.read_excel('sample_data/MovieDataset1.xlsx')
movies.head()

,TITLE,Description
0,21,"Ben Campbell, a mathematics major at the Massa..."
1,10 Things I Hate About You,"Cameron James, a new student at Padua High Sch..."
2,101 Dalmatians(1996),American video game designer Roger Dearly live...
3,12 Angry Men,"On a hot summer day, a jury in the New York Co..."
4,12 Years a Slave,Solomon Northup is a free African-American man...


In [7]:
#Preprocess the text with first splitting the sentence into seperate words and set a maximum number of 1024 words
def preprocess_text(text, max_words=1024):
    if isinstance(text, float) and pd.isnull(text):
        return ''

    # Split text into words and limit to max_words
    words = text.split()[:max_words]

    # Join words back into a single string
    text = ' '.join(words)

    # Remove newline, carriage return, and tab characters
    regex = re.compile(r'[\n\r\t]')
    text = regex.sub(" ", text)

    # Remove extra spaces
    text = re.sub(' +', ' ', text).strip()

    return text

In [8]:
#Apply Preprocessing to Description column
movies['Description'] = movies['Description'].apply(preprocess_text)

In [9]:
#Check if GPU is available or else use CPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [10]:
#Training Data
plots = []
for i in movies.Description:
    try:
        k = i.split(" ")
        if 1110 > len(k) > 591:
            plots.append(i)
    except:
        continue

In [11]:
plot_list = movies.Description.tolist()

def split_list(lst, batch):
    for i in range(0, len(lst), batch):
        yield lst[i:i + batch]

batch = 32
batches = list(split_list(plot_list, batch))

In [12]:
#BART model specifications
model = BartForConditionalGeneration.from_pretrained('sshleifer/distilbart-cnn-12-3')
model.to(device)
model.eval()
tokenizer = BartTokenizer.from_pretrained('sshleifer/distilbart-cnn-12-3')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.73k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

In [13]:
#Generate summaries using BART
all_summaries = []
for batch in batches:
    with torch.no_grad():
        inputs = tokenizer(batch,
                           max_length=1024,
                           padding=True,
                           truncation=True,
                           return_tensors='pt')

        summary_ids = model.generate(inputs['input_ids'].to(device), max_length=500, early_stopping=True).to('cpu')

        all_summaries+= [txt.strip() for txt in tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)]
        del inputs, summary_ids
        torch.cuda.empty_cache()
movies['Plot Summary(bart)'] = all_summaries

In [14]:
#Save generated summaries into an excel file
movies=movies.to_excel('MovieDataset2.xlsx', index=False)

In [15]:
#Display generated bart summaries
op=pd.read_excel("MovieDataset2.xlsx")
op.head()

,TITLE,Description,Plot Summary(bart)
0,21,"Ben Campbell, a mathematics major at the Massa...","Ben Campbell, a mathematics major at the Massa..."
1,10 Things I Hate About You,"Cameron James, a new student at Padua High Sch...",Michael Eckman warns sophomore Bianca Stratfor...
2,101 Dalmatians(1996),American video game designer Roger Dearly live...,American video game designer Roger Dearly live...
3,12 Angry Men,"On a hot summer day, a jury in the New York Co...",The 18-year-old boy is accused of killing his ...
4,12 Years a Slave,Solomon Northup is a free African-American man...,Slave trader Theophilus Freeman gives Northup ...


In [16]:
#Generate summaries for movies not in the dataset by referring to wikipedia and using t5 model

# Initialize NLTK tokenizer and download punkt
nltk.download('punkt')

# Initialize T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# Initialize Rouge for ROUGE-L scores
rouge = Rouge()

# Function to get plot summary from dataset
def fetch_movie_plot_from_dataset(movie_title, op):
    plot_summary = op.loc[op['TITLE'] == movie_title, 'Plot Summary(bart)'].values
    return plot_summary[0] if len(plot_summary) > 0 else None

# Function to get plot summary from Wikipedia
def fetch_movie_plot_from_wikipedia(movie_title):
    try:
        page = wikipedia.page(movie_title)
        content = page.content
        plot_section = find_plot_section(content)
        return plot_section
    except (wikipedia.exceptions.PageError, wikipedia.exceptions.DisambiguationError):
        return None

# Function to extract plot section from content
def find_plot_section(content):
    start_idx = content.find("== Plot ==")
    if start_idx == -1:
        start_idx = content.find("== Plot== ")
    if start_idx != -1:
        end_idx = content.find("\n==", start_idx + len("== Plot =="))
        if end_idx == -1:
            plot_section = content[start_idx:].strip()
        else:
            plot_section = content[start_idx:end_idx].strip()
        return plot_section
    else:
        return None

# Function to generate summary using T5 model
def generate_summary(content, max_length=1024):
    if content:
        inputs = tokenizer.encode("summarize: " + content, return_tensors="pt", max_length=1024, truncation=True)
        summary_ids = model.generate(inputs, max_length=900, min_length=100, length_penalty=5.0, num_beams=10)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary
    else:
        return None

# Function to get plot summary and content
def get_plot_summary(movie_title, op):
    summary = op.loc[op['TITLE'] == movie_title, 'Plot Summary(bart)']
    if not summary.empty:
        content = op.loc[op['TITLE'] == movie_title, 'Description'].values[0]
        return summary.values[0], content  # Return both summary and content
    else:
        wikipedia_summary = fetch_movie_plot_from_wikipedia(movie_title)
        if wikipedia_summary:
            t5_summary = generate_summary(wikipedia_summary)
            return t5_summary, wikipedia_summary
        else:
            return f"Plot summary for '{movie_title}' not found in dataset or Wikipedia.", None

# Testing and evaluation
# ROUGE-Recall-Oriented Understudy for Gisting Evaluation

# Main loop(user input)
while True:
    movie_title = input("\nEnter title of the movie to summarize (type 'exit' to quit):\n")
    if movie_title.lower() == 'exit':
        print("\nExited")
        break
    else:
        summary, content = get_plot_summary(movie_title, op)
        wrapped_summary = textwrap.fill(summary, width=90) #To fit within the screen
        if isinstance(summary, str):
            print("Generated Summary:\n", wrapped_summary)
            if content and isinstance(content, str):
                # Calculate ROUGE-1 scores
                rouge_scores = rouge.get_scores(summary, content, avg=True)
                print("\nROUGE-1 Scores:")
                print(f"Precision: {rouge_scores['rouge-1']['p']}")
                print(f"Recall: {rouge_scores['rouge-1']['r']}")
                print(f"F1 Score: {rouge_scores['rouge-1']['f']}")
                overall_rouge1 = (rouge_scores['rouge-1']['p'] + rouge_scores['rouge-1']['r'] + rouge_scores['rouge-1']['f']) / 3
                print(f"\nOverall ROUGE-1 Score: {overall_rouge1}")
            else:
                print("\nUnable to calculate ROUGE-1 scores because content is not available or not a string.")
        else:
            print(f"{summary}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]


Enter title of the movie to summarize (type 'exit' to quit):
12 Angry Men
Generated Summary:
 The 18-year-old boy is accused of killing his abusive father. A neighbor testified to
witnessing the defendant stab his father, from her window, through the windows of a
passing train. In a preliminary vote, all jurors vote "guilty" except Juror 8, who
believes there should be some discussion before the verdict is made. Juror 3, infuriated,
argues with and tries to attack Juror.

ROUGE-1 Scores:
Precision: 1.0
Recall: 0.1569767441860465
F1 Score: 0.2713567815742027

Overall ROUGE-1 Score: 0.4761111752534164

Enter title of the movie to summarize (type 'exit' to quit):
chhichhore
Generated Summary:
 anirudh "anni" pathak is a divorced middle-aged man living with his teenage son. he is
awaiting the results of his entrance examination in the hope of enrolling in the iit-jEE.
he deliberately slips off the balcony to commit suicide, but survives, although in a fatal
state. Anni and his ex-wife may

In [17]:
#Summaries can also be generated using PEGASUS model in this way:

# Initialize Pegasus tokenizer and model
model_name = "google/pegasus-large"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

# Function to get plot summary and content
def get_plot_summary(movie_title, op):
    summary = op.loc[op['TITLE'] == movie_title, 'Plot Summary(bart)']
    if not summary.empty:
        content = op.loc[op['TITLE'] == movie_title, 'Description'].values[0]
        return summary.values[0], content  # Return both summary and content
    else:
        wikipedia_summary = fetch_movie_plot_from_wikipedia(movie_title)
        if wikipedia_summary:
            pegasus_summary = generate_summary(wikipedia_summary)
            return pegasus_summary, wikipedia_summary
        else:
            return f"Plot summary for '{movie_title}' not found in dataset or Wikipedia.", None

# Main loop for testing and evaluation
while True:
    movie_title = input("\nEnter title of the movie to summarize (type 'exit' to quit):\n")
    if movie_title.lower() == 'exit':
        print("\nExited")
        break
    else:
        summary, content = get_plot_summary(movie_title, op)
        wrapped_summary = textwrap.fill(summary, width=90)
        if isinstance(summary, str):
            print("Generated Summary:\n", wrapped_summary)
            if content and isinstance(content, str):
                # Calculate ROUGE-1 scores
                rouge_scores = rouge.get_scores(summary, content, avg=True)
                print("\nROUGE-1 Scores:")
                print(f"Precision: {rouge_scores['rouge-1']['p']}")
                print(f"Recall: {rouge_scores['rouge-1']['r']}")
                print(f"F1 Score: {rouge_scores['rouge-1']['f']}")
                overall_rouge1 = (rouge_scores['rouge-1']['p'] + rouge_scores['rouge-1']['r'] + rouge_scores['rouge-1']['f']) / 3
                print(f"\nOverall ROUGE-1 Score: {overall_rouge1}")
            else:
                print("\nUnable to calculate ROUGE-1 scores because content is not available or not a string.")
        else:
            print(f"{summary}")


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/3.09k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/260 [00:00<?, ?B/s]


Enter title of the movie to summarize (type 'exit' to quit):
inception(2010 film)
Generated Summary:
 He offers to hire Cobb for the ostensibly impossible job of implanting an idea into a
person's subconscious; performing "inception" on Robert Fischer, the son of Saito's
competitor Maurice Fischer, with the idea to dissolve his father's company. Ariadne is
tasked with designing the dream's architecture, something Cobb himself cannot do for fear
of being sabotaged by his mind's projection of his late wife, Mal. Cobb tells Ariadne that
he and Mal entered Limbo while experimenting with dream-sharing, experiencing fifty years
in one night due to the time dilation with reality. Cobb persuades Robert that Browning
has kidnapped him to stop the inception and that Cobb is a defensive projection, leading
Robert to another third level deeper as part of a ruse to enter Robert's subconscious. In
the third level, Cobb tells Ariadne that he and Mal entered Limbo while experimenting with
dream-shari

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Generated Summary:
 Plot summary for 'got' not found in dataset or Wikipedia.

Unable to calculate ROUGE-1 scores because content is not available or not a string.

Enter title of the movie to summarize (type 'exit' to quit):
frgh
Generated Summary:
 Plot summary for 'frgh' not found in dataset or Wikipedia.

Unable to calculate ROUGE-1 scores because content is not available or not a string.

Enter title of the movie to summarize (type 'exit' to quit):
exit

Exited
